<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/cogmen_feature_foramt19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import csv
import pandas as pd
import os
import pickle
from tqdm import tqdm
annotation_root= '/content/gdrive/MyDrive/emotion_recognition/dataset/KEMDy19/annotation'
txt_label_zip_toteval_root='/content/gdrive/MyDrive/emotion_recognition/dataset/2019cogmen_format__.pkl'
ambig_annotation=0
no_file=0
## formating features dim 9 list

features=[]
sess_ids=[]
sess_speakers=dict()
sess_labels=dict()
sess_text=dict()
sess_audio=dict()
sess_visual=dict()
sess_sentence=dict()
sess_dir=dict()
sess_train=[]
sess_test=[]

if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    features=pickle.load(f)
else:
  
  for i, annotation in tqdm(enumerate(os.listdir(annotation_root))):
    all_txts=[]
    print('annotaion file : (%d / %d)'%(i,len(os.listdir(annotation_root))))
    session_file = pd.read_csv(os.path.join(annotation_root,annotation)) 
    # print(session_file) 
    segments=session_file['Segment ID']
    session_file.loc[(session_file['Total Evaluation'] == "fear"), 'Total Evaluation'] = 0  #공포 => 0
    session_file.loc[(session_file['Total Evaluation'] == "surprise"), 'Total Evaluation'] = 1  #놀람 => 1
    session_file.loc[(session_file['Total Evaluation'] == "angry"), 'Total Evaluation'] = 2  #분노 => 2
    session_file.loc[(session_file['Total Evaluation'] == "sad"), 'Total Evaluation'] = 3  #슬픔 => 3
    session_file.loc[(session_file['Total Evaluation'] == "neutral"), 'Total Evaluation'] = 4  #중립 => 4
    session_file.loc[(session_file['Total Evaluation'] == "happy"), 'Total Evaluation'] = 5  #행복 => 5
    session_file.loc[(session_file['Total Evaluation'] == "disgust"), 'Total Evaluation'] = 6  #혐오 => 6
    
    
    
    
    # print(segments)
    ##segments == utterence  
    for j in range(1,len(segments)):
       #index starts from 1
      label= session_file['Total Evaluation'][j]
      if type(label) !=int:
        ambig_annotation+=1
        continue
      f_name=segments[j]
      gender=f_name.split('_')[-1][0]
      
      # print("f_name:",f_name)
      sess_num=f_name.split('_')[0][-2:]
      sc_pro_num=f_name.split('_')[0]+'_'+f_name.split('_')[1]

      ###texts
      file_loc='/content/gdrive/MyDrive/emotion_recognition/dataset/KEMDy19/wav/Session%s/%s/%s.txt'%(sess_num,sc_pro_num,f_name)
      # print(file_loc)
      if not os.path.isfile(file_loc):
        print("no file location :",file_loc)
        no_file+=1
        continue
      with open(file_loc) as f:
        lines = f.readlines()[0]
      if '\n' in lines or'/'in lines :
        # print('lines',lines)
        new_words=lines.replace('\n','')
        # print('new_words',new_words)
        new_words=new_words.replace('o/','')
        new_words=new_words.replace('I/','')
        new_words=new_words.replace('b/','')
        new_words=new_words.replace('u/','')
        new_words=new_words.replace('l/','')
        new_words=new_words.replace('n/','')
        new_words=new_words.replace('N/','')
        new_words=new_words.replace('s/','')
        new_words=new_words.replace('c/','')
        new_words=new_words.replace('(','')
        new_words=new_words.replace(')','')
        new_words=new_words.replace('+',' ')
        lines=new_words





     
      if sc_pro_num not in sess_ids:
        sess_ids.append(sc_pro_num)
        sess_speakers[sc_pro_num]=[]
        sess_labels[sc_pro_num]=[]
        sess_text[sc_pro_num]=[]
        sess_audio[sc_pro_num]=[]
        sess_visual[sc_pro_num]=[]
        sess_sentence[sc_pro_num]=[] 
        sess_dir[sc_pro_num]=[]
        if int(sess_num) ==19 or int(sess_num)==20:
          sess_test.append(sc_pro_num)
        else:
          sess_train.append(sc_pro_num)

      sess_labels[sc_pro_num].append(label)
      sess_speakers[sc_pro_num].append(gender)
      sess_dir[sc_pro_num].append(f_name)
      # print('labels',labels,type(labels))
      # print('stcs',lines,type(lines))
      # sess_labels[sc_pro_num]=file_labels
      sess_sentence[sc_pro_num].append(lines)
      
      # print('1 file ends')
      
      # print("ids",sess_ids)
      # print("speakers",sess_speakers)
      # print("labels",sess_labels)
      # print("text",sess_text)
      # print("audio",sess_audio)
      # print("visual",sess_visual)
      # print("stcs",sess_sentence)
      # print("tr",sess_train)
      # print("te",sess_test)
      # print(sess_num,sc_pro_num)
      
    #  /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.txt
      
      # print(all_txts) 
  # print("ids",sess_ids)
  # print("speakers",sess_speakers)
  # print("labels",sess_labels)
  # print("text",sess_text)
  # print("audio",sess_audio)
  # print("visual",sess_visual)
  # print("stcs",sess_sentence)
  # print("tr",sess_train)
  # print("te",sess_test)
  
  features.append(sess_ids)
  features.append(sess_speakers)
  features.append(sess_labels)
  features.append(sess_text)
  features.append(sess_audio)
  features.append(sess_visual)
  features.append(sess_sentence)
  features.append(sess_train)
  features.append(sess_test)
  features.append(sess_dir)
  # print(features)
  with open(txt_label_zip_toteval_root,'wb') as f:
    pickle.dump(features,f)
# print("data_list length :",len(data_list))
# print("sample : ",data_list[0])
print('ambig_annotation:',ambig_annotation)  
print('no_file',no_file)  
# print(len(all_txts))





In [33]:
import os
import pickle
txt_label_zip_toteval_root='/content/gdrive/MyDrive/emotion_recognition/dataset/2019cogmen_format_feature_final.pkl'
if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    features=pickle.load(f)

In [35]:
print(features[0])
# print((features[9]))
print(len(features[4].values()))
# for script in features[0]:
#   print(len(features[1][script]))
#   print(len(features[2][script]))
#   print(len(features[3][script]))
#   print(len(features[4][script]))
#   print(len(features[6][script]),'\n')
print(len(features[1]['Sess19_script02']))
print(len(features[2]['Sess19_script02']))
print(len(features[6]['Sess19_script02']))

print((features[3]['Sess19_script02'][0].shape))
print((features[4]['Sess19_script02'][0].shape))

# with open('/content/gdrive/MyDrive/emotion_recognition/dataset/2019cogmen_format_feature_final.pkl','wb') as f:
#   pickle.dump(features[:8],f)

# print(len(features[3]['Sess03_script01']))
# print(len(features[4]['Sess03_script01']))
# for i in range(10):
#   print(len(features[i]))

['Sess01_script01', 'Sess01_script02', 'Sess01_script03', 'Sess01_script04', 'Sess01_script05', 'Sess01_script06', 'Sess01_impro01', 'Sess01_impro02', 'Sess01_impro03', 'Sess01_impro04', 'Sess02_script01', 'Sess02_script02', 'Sess02_script03', 'Sess02_script04', 'Sess02_script05', 'Sess02_script06', 'Sess02_impro01', 'Sess02_impro02', 'Sess02_impro03', 'Sess02_impro04', 'Sess05_script01', 'Sess05_script02', 'Sess05_script03', 'Sess05_script04', 'Sess05_script05', 'Sess05_script06', 'Sess05_impro01', 'Sess05_impro02', 'Sess05_impro03', 'Sess05_impro04', 'Sess08_script01', 'Sess08_script02', 'Sess08_script03', 'Sess08_script04', 'Sess08_script05', 'Sess08_script06', 'Sess08_impro01', 'Sess08_impro02', 'Sess08_impro03', 'Sess08_impro04', 'Sess09_script01', 'Sess09_script02', 'Sess09_script03', 'Sess09_script04', 'Sess09_script05', 'Sess09_script06', 'Sess09_impro01', 'Sess09_impro02', 'Sess09_impro03', 'Sess09_impro04', 'Sess10_script01', 'Sess10_script02', 'Sess10_script03', 'Sess10_scri

In [2]:
#### feature extractor
!pip install transformers
!pip install sklearn
!pip install scipy
!pip install pandas

!pip install audtorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=5a6c9216c0e52f7d8dfc31ecf1b93888a9badb565162c0a5fd77bda311995ede
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.o

In [20]:
%cd /content/gdrive/MyDrive/emotion_recognition/2022reference/

!python multimodal_feature_extractor_cogmen.py --batch_size 1
# !python multimodal_feature_extractor.py --dataset 19 --dataset_dir '/content/gdrive/MyDrive/emotion_recognition/dataset/KEMDy19'

/content/gdrive/MyDrive/emotion_recognition/2022reference
2023-04-05 04:48:42.908079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
device: cuda
1
2
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAI

In [ ]:
|